In [1]:
from graph_datasets.coco import CocoStuffGraphDataset
coco_graph = CocoStuffGraphDataset()
data = coco_graph[1]
data

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since limingcv/Captioned_COCOStuff couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/rupertmenneer/.cache/huggingface/datasets/limingcv___captioned_coco_stuff/default/0.0.0/98bb1b8cbce93f9946f79c79e837e4b8502d094d (last modified on Tue Sep  3 17:19:48 2024).


RelaxedHeteroData(
  image=[1, 3, 256, 256],
  mask=[1, 1, 256, 256],
  image_node={
    x=[1024, 1],
    pos=[1024, 2],
  },
  class_node={
    x=[6, 183],
    pos=[6, 2],
  },
  (class_node, class_edge, class_node)={ edge_index=[2, 30] },
  (class_node, class_to_image, image_node)={ edge_index=[2, 1023] }
)

In [2]:
from training.networks_edm2_hignn import Precond
precond = Precond(32, 3, gnn_metadata = data.metadata())

In [5]:
import torch
images = torch.randn(1, 3, 32, 32)
rnd_normal = torch.randn([images.shape[0], 1, 1, 1], device=images.device)
sigma = (rnd_normal).exp()

data = coco_graph[1]
out = precond(images, sigma, data)
out.shape

torch.Size([1, 3, 32, 32])

In [12]:
from training.training_loop_hignn import training_loop

training_loop(device='cpu')

Loading dataset...
Setting up encoder...
ref_image torch.Size([1, 3, 256, 256])


RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 1